## Packages and Assets

In [1]:
import pandas as pd
import re
import string
import nltk
from sklearn.manifold import TSNE
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from gensim.models import Word2Vec

In [2]:
df_pp = pd.read_csv('../../assets/data/df_pp.csv')
df_pp

,title,tags,link,label
0,economia brasileira recuperado gradualmente di...,"['Copom', 'taxa de juros', 'taxa selic']",https://www.sunoresearch.com.br/noticias/econo...,0
1,gasolina vai ficar cara veja politica precos p...,"['Combustíveis', 'Diesel', 'Etanol', 'Gasolina...",https://www.moneytimes.com.br/gasolina-vai-fic...,1
2,vale vale3 ubs recomenda vender acao rapido po...,"['Ações', ""ADR's"", 'Bancos', 'China', 'Comprar...",https://www.moneytimes.com.br/vale-vale3-ubs-r...,2
3,vale entrega plano desmonte barragem barao coc...,['Vale (VALE3)'],https://www.suno.com.br/noticias/vale-plano-de...,2
4,black friday corretoras incentivo abertura con...,"['Black Friday 2020', 'Black Friday das corret...",https://www.sunoresearch.com.br/noticias/black...,0
...,...,...,...,...
18007,acao disparou <NUM> cento estreia <NUM> indica...,"['Ações', 'Ativa Investimentos', 'Banco Inter'...",https://www.moneytimes.com.br/acao-que-disparo...,2
18008,balanca comercial superavit <NUM> bilhoes feve...,"['balança comercial', 'exportações', 'importaç...",https://www.sunoresearch.com.br/noticias/balan...,0
18009,agenda dia oi riachuelo banco inter locaweb ca...,"['agenda', 'Banco Inter', 'BIDI11', 'Caixa Eco...",https://www.sunoresearch.com.br/noticias/agend...,0
18010,petrobras arremata producao dois campos bacia ...,"['Empresas', 'Petrobras']",https://www.moneytimes.com.br/petrobras-arrema...,1


## Functions

## Loading Word2vec Model

## Word2Vec Model

In [3]:
hasModel = False
if not hasModel:

    tokenized_corpus = []
    for title in df_pp['title']:
        tokenized_corpus.append(title.split())
    model = Word2Vec(sentences=tokenized_corpus, vector_size=300, window=20, min_count=5,workers =4)
    model.save("../../assets/lstm_assets/word2vec.model")
else:
    print("Model Exists!")
    model = Word2Vec.load("../../assets/lstm_assets/word2vec.model")
    print(model.wv.most_similar('queda'))


In [4]:
model.wv.get_normed_vectors()

array([[-0.03236833,  0.03023816,  0.06029034, ..., -0.08823165,
         0.0872182 , -0.02627683],
       [-0.00204058,  0.0962977 ,  0.03250765, ..., -0.02544235,
         0.02294619,  0.00153643],
       [-0.00075033,  0.01500599,  0.08460046, ..., -0.06417342,
         0.10984575, -0.05269717],
       ...,
       [-0.00301698,  0.09577885, -0.01550451, ..., -0.05175942,
         0.02537947, -0.01378746],
       [-0.01466258,  0.07552481,  0.03784581, ..., -0.06458158,
         0.05610614, -0.02230895],
       [-0.02195597,  0.00314152,  0.06380174, ..., -0.08491288,
         0.07712409, -0.02078664]], dtype=float32)

In [5]:
model.wv.index_to_key

['<NUM>',
 'petrobras',
 'cento',
 'vale',
 'diz',
 'itau',
 'acoes',
 'petr4',
 'ibovespa',
 'alta',
 'queda',
 'brasil',
 'apos',
 'minerio',
 'sobre',
 'carteira',
 'vale3',
 'bi',
 'ferro',
 'venda',
 'milhoes',
 'semana',
 'dividendos',
 'lucro',
 'bilhoes',
 'petroleo',
 'dolar',
 'pode',
 'mercado',
 'acao',
 'banco',
 'acordo',
 'veja',
 'bolsonaro',
 'fecha',
 'producao',
 'preco',
 'governo',
 'itub4',
 'gasolina',
 'nesta',
 'anuncia',
 'eua',
 'vai',
 'nova',
 'bolsa',
 'dia',
 'sobe',
 'precos',
 'brumadinho',
 'abre',
 'xp',
 'coronavirus',
 'cai',
 'novo',
 'mi',
 'china',
 'compra',
 'diesel',
 'bancos',
 'deve',
 'registra',
 'trimestre',
 'aprova',
 'btg',
 'gas',
 'presidente',
 'dalian',
 'maior',
 'oferta',
 'radar',
 'participacao',
 'empresas',
 'mil',
 'inicia',
 'bradesco',
 'recomendada',
 'investimentos',
 've',
 'combustiveis',
 'confira',
 'segundo',
 'semanal',
 'pontos',
 'troca',
 'liquido',
 'eleva',
 'reduz',
 'ano',
 'pib',
 'premenos',
 'ipo',
 'br',

In [6]:
word_embeddings = []
vocab = []

for i, word in enumerate(model.wv.index_to_key):
    vocab.append(word)
    word_embeddings.append(model.wv[word])

In [7]:
emb_df = pd.DataFrame(word_embeddings)
emb_df = emb_df.assign(nome=vocab)
emb_df

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,nome
0,-0.168238,0.157166,0.313365,0.357736,-0.401158,-0.410387,0.485782,0.727607,-0.253907,-0.416206,...,0.190522,0.598968,-0.409415,0.776612,0.187728,0.167323,-0.458593,0.453326,-0.136577,<NUM>
1,-0.011100,0.523845,0.176837,0.533204,0.062346,0.199473,0.141219,0.709443,0.764909,-0.557238,...,0.490605,0.552759,0.524615,0.640188,0.557422,-0.017307,-0.138403,0.124824,0.008358,petrobras
2,-0.005005,0.100099,0.564339,0.299671,-0.418595,-0.392340,0.957735,0.670811,-0.248862,-0.658701,...,0.474809,0.715989,-0.195722,0.786600,0.281371,0.240041,-0.428077,0.732741,-0.351524,cento
3,-0.427080,-0.106793,-0.064216,0.114558,0.066548,-0.560485,-0.348762,0.581624,-0.104919,0.168650,...,0.016313,-0.002347,-0.393555,0.090410,0.022970,-0.105865,-0.387094,0.093603,0.347530,vale
4,-0.077757,0.246810,-0.003233,0.377013,0.044431,-0.022526,0.045652,0.578031,0.594330,-0.263887,...,0.430558,0.418529,0.206729,0.312358,0.399935,-0.005026,-0.117615,0.103631,0.132893,diz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3429,-0.004428,0.034264,-0.000436,0.019951,0.007822,-0.047770,0.023769,0.075056,0.025227,-0.027746,...,0.047084,0.034527,0.010385,0.036306,0.043360,-0.002662,-0.022751,0.012984,-0.011608,sente
3430,-0.003093,0.045650,0.003019,0.024420,0.008195,-0.066893,0.020750,0.089837,0.023538,-0.026120,...,0.048199,0.043542,0.000799,0.048957,0.051348,-0.005324,-0.026963,0.011436,-0.010457,big
3431,-0.001058,0.033581,-0.005436,0.021415,0.005841,-0.048823,0.012083,0.071119,0.025709,-0.014731,...,0.041705,0.028832,0.009880,0.031835,0.041015,-0.005824,-0.018147,0.008898,-0.004834,tentar
3432,-0.003932,0.020255,0.010150,0.014465,0.000675,-0.032429,0.029480,0.045026,0.004291,-0.016730,...,0.026421,0.024662,0.001112,0.021668,0.029526,0.004431,-0.017320,0.015047,-0.005983,sinalizacao


In [ ]:
tsne = TSNE(n_components=2,perplexity=40, n_iter=2500, random_state=23, learning_rate=100,metric='euclidean',init='pca')
X_tsne = tsne.fit_transform(np.array(word_embeddings))
X_tsne[1:4,:]
emb_df['x'] = X_tsne[:,0]
emb_df['y'] = X_tsne[:,1]
fig = px.scatter(emb_df,x='x',y='y',hover_data={'x':False, 'y':False,'nome':True})
fig.show()